# Anthropic Agent

### State Diagram (Agent View)
```mermaid
stateDiagram-v2
direction TB
    INIT --> IS_TOOL_CALL
    IS_TOOL_CALL --> CHAT: condition_false
    IS_TOOL_CALL --> TOOL_USE: condition_true
    
    CHAT --> IS_TERMINATE
    TOOL_USE --> IS_TERMINATE
    
    IS_TERMINATE --> IS_TOOL_CALL: condition_false
    IS_TERMINATE --> FINAL: condition_true
```



## Setup: Create Agent

In [ ]:
import os
os.environ["LOG_LEVEL"] = "WARNING"

In [1]:
import os
from gai.lib.constants import DEFAULT_GUID
from gai.asm.agents import ToolUseAgent
from gai.mcp.client.mcp_client import McpAggregatedClient
from gai.lib.config import config_helper
from gai.messages import FileMonologue
from gai.lib.tests import make_local_tmp

# Configure Monologue

here = make_local_tmp()
file_path = os.path.join(here, "monologue.json")
monologue = FileMonologue(agent_name="ToolUseAgent",file_path=file_path)

# Configure MCP

aggregated_client = McpAggregatedClient(["mcp-pseudo","mcp-time"])
tools = await aggregated_client.list_tools()

# Configure LLM

llm_config = config_helper.get_client_config(
    {
        "client_type": "anthropic",
        "model": "claude-sonnet-4-20250514",
        "extra": {
            "max_tokens": 32000,
            "temperature": 0.7,
            "top_p": 0.95,
            "tools": True,
            "stream": True,
        },
    }
)


def print_chunk(chunk):
    """
    Helper function to print the chunk of data received from the agent.
    """
    if chunk:
        if isinstance(chunk, str):
            print(chunk, end="", flush=True)
        else:
            if isinstance(chunk, list):
                for item in chunk:
                    if item.get("name"):
                        print(f'Tool: "{item["name"]}"')
                    if item.get("input"):
                        inputs = item.get("input")
                        if isinstance(inputs, dict):
                            for key, value in inputs.items():
                                if isinstance(value, str):
                                    if len(value) > 100:
                                        print(
                                            f"\tInput: {key} = {value[:100]}... (truncated)"
                                        )
                                    else:
                                        print(f"\tInput: {key} = {value}")

---

## Scenario 1: Good Flow

Let us construct a hypothetical conversation below.

In [2]:
from gai.messages import FileDialogue, MessagePydantic

# Create an artificial dialogue history for testing

messages = [
    MessagePydantic(
        **{
            "id": "b1e5f98c-f6eb-47de-a6e2-387510d970f9",
            "header": {
                "sender": "User",
                "recipient": "Sara",
                "timestamp": 1751308157.270983,
                "order": 0,
            },
            "body": {
                "type": "chat.send",
                "dialogue_id": "00000000-0000-0000-0000-000000000000",
                "round_no": 0,
                "step_no": 0,
                "role": "user",
                "content": "It is a very nice weather in Singapore right now.",
            },
        }
    ),
    MessagePydantic(
        **{
            "id": "abbc7961-45dc-4973-aaf4-a6224ed35d37",
            "header": {
                "sender": "Sara",
                "recipient": "User",
                "timestamp": 1751308167.3488164,
                "order": 1,
            },
            "body": {
                "type": "chat.reply",
                "dialogue_id": "00000000-0000-0000-0000-000000000000",
                "round_no": 0,
                "step_no": 1,
                "chunk_no": 10,
                "chunk": "<eom>",
                "role": "assistant",
                "content": "Yes, it is! The weather in Singapore is typically warm and humid, with occasional rain showers. It's a great time to enjoy outdoor activities or relax indoors with a cool drink. How can I assist you today?",
            },
        }
    ),
]
file_path = os.path.join(here, f"{DEFAULT_GUID}.json")
dialogue = FileDialogue(messages=messages, file_path=file_path)
recap = dialogue.extract_recap()


### a) Start Agent

Extract the conversation as recap from dialogue and pass it to the agent. Agent can infer the timezone from the recap.

In [3]:

agent = ToolUseAgent(
    agent_name="ToolUseAgent",
    llm_config=llm_config,
    aggregated_client=aggregated_client,
    monologue=monologue
)

user_message = "What is the current time here?"

resp = agent.start(user_message=user_message, recap=recap)
async for chunk in resp:
    print_chunk(chunk)


I'll help you get the current time in Singapore. Let me check that for you.
Tool: "current_time"
	Input: format = YYYY-MM-DD HH:mm:ss
	Input: timezone = Asia/Singapore


### b) Tool Call

Call the MCP and return the result. Also note that the agent is stateless, we can use a new instance to continue the conversation.


In [4]:
agent = ToolUseAgent(
    agent_name="ToolUseAgent",
    llm_config=llm_config,
    aggregated_client=aggregated_client,
    monologue=monologue,
)

resp = agent.resume()
async for chunk in resp:
    print_chunk(chunk)
print(f"\ncompleted state: {agent.fsm.state}")
assert agent.fsm.state == "IS_TERMINATE"

The current time in Singapore is **11:00:20 AM on July 31, 2025**. It's a lovely morning time to enjoy that nice weather you mentioned!

completed state: IS_TERMINATE


### c) End Conversation

Once the agent has finished its task, calling resume() will fail. To continue, you can use resume(user_message) to update the conversation or start a new one with start(user_message).

In [5]:
from gai.asm.agents.tool_use_agent import AutoResumeError
try:
    resp = agent.resume()
    async for chunk in resp:
        print_chunk(chunk)
except AutoResumeError as e:
    print(f"Test passed: conversation is over. {str(e)}")

    # Save the user message and assistant message to the dialogue at end of the conversation
    assistant_message = agent.fsm.state_bag["get_assistant_message"]()
    dialogue.add_user_message(recipient="Sara", content=user_message)
    dialogue.add_assistant_message(sender="Sara", chunk="<eom>", content=assistant_message)

Test passed: conversation is over. ToolUseAgent.resume: Cannot resume() as agent has completed its task. Either resume(user_message) to update the task or start a new task with start_async(user_message).


### d) Update conversation

In [7]:
resp = agent.resume("Tell me a one sentence story.")
async for chunk in resp:
    print_chunk(chunk)

Here's a one sentence story for you:

As the tropical rain began to fall in Singapore at 11 AM, Maya discovered that the old umbrella she found in her grandmother's attic could transport her to any place she had ever dreamed of visiting.


### e) Show monologue

See the monologue for details at `tmp/monologue.json`

In [10]:
import json
from gai.messages import message_helper

# Show the monologue
print("\n───────────────────────── MONOLOGUE START ─────────────────────────")
messages = agent.monologue.list_chat_messages()
for message in messages:
    print(json.dumps(message, indent=4))
print("───────────────────────── MONOLOGUE END ─────────────────────────\n")

# Print memory size
mem_size=message_helper.get_messages_length(messages)
print("Total char size=", mem_size)


───────────────────────── MONOLOGUE START ─────────────────────────
{
    "role": "user",
    "content": "\n            Your name is ToolUseAgent within the context of this conversation and you will always respond as such.\n            Do not refer to yourself as an AI or a bot or confuse your name with other agents.\n           \n            You may respond to my following message using the context you have learnt.\n            \n            What is the current time here?\n\n            Here is a recap of the conversation:\n            User: It is a very nice weather in Singapore right now.\nSara: Yes, it is! The weather in Singapore is typically warm and humid, with occasional rain showers. It's a great time to enjoy outdoor activities or relax indoors with a cool drink. How can I assist you today?\n            \n            \n            You may ask me for more information if you need to clarify my request but ask just enough to get the information you need to get started.\n       

### f) Show dialogue

In [8]:
for msg in dialogue.list_messages():
    print(f"{msg.header.sender}: {msg.body.content}")

User: It is a very nice weather in Singapore right now.
Sara: Yes, it is! The weather in Singapore is typically warm and humid, with occasional rain showers. It's a great time to enjoy outdoor activities or relax indoors with a cool drink. How can I assist you today?
User: Sara, What is the current time here?
Sara: The current time in Singapore is **10:33:13 AM on July 31st, 2025**. It's a lovely morning time to be enjoying that nice weather you mentioned!


---

## Scenario 2: Agent interrupt user for input

We will invite the agent to ask questions in this scenario.

### a) Start Agent

In [8]:
agent = ToolUseAgent(
    agent_name="ToolUseAgent",
    llm_config=llm_config,
    aggregated_client=aggregated_client,
    monologue=monologue,
)

We know the agent is expecting response from the user when it calls "user_input" tool.

In [9]:
user_message = "What is the current time? Please ask if you need more information."

resp = agent.start(user_message=user_message)
async for chunk in resp:
    print_chunk(chunk)


I'd be happy to help you get the current time! To provide you with the most accurate time, I need to know a couple of details:
Tool: "user_input"


### b) This will throw error unless user input is provided

In [ ]:
from gai.asm.agents.tool_use_agent import PendingUserInputError
try:
    resp = agent.resume()
    async for chunk in resp:
        print_chunk(chunk)
except PendingUserInputError as e:
    assert "pending user input" in str(e)
    print(f"Error occurred: {str(e)}")
    print("This is expected as the agent is waiting for user input.")


ERROR    ToolUserAgent.resume: ToolUseAgent._resume_async: pending user input

Error occurred: ToolUseAgent._resume_async: pending user input
This is expected as the agent is waiting for user input.


### c) Can resume normally after user input

In [ ]:
# IS_TOOL_CALL -> TOOL_USE
resp = agent.resume("Use SGT")
async for chunk in resp:
    print_chunk(chunk)



Tool: "current_time"
	Input: format = YYYY-MM-DD HH:mm:ss
	Input: timezone = Asia/Singapore


---

## Scenario 3: User interrupt agent with adhoc input

In this scenario, the user tries to distract the agent by interrupting it with an adhoc input.

### a) Start Agent


In [10]:
agent = ToolUseAgent(
    agent_name="ToolUseAgent",
    llm_config=llm_config,
    aggregated_client=aggregated_client,
    monologue=monologue,
)

# Start the agent

user_message = "What is the current time? Please ask if you need more information."

resp = agent.start(user_message=user_message)
async for chunk in resp:
    print_chunk(chunk)

I'd be happy to get the current time for you! To provide the most accurate information, I need to know a couple of details:
Tool: "user_input"


### b) User interrupt agent

Should be able to interrupt the agent and continue with original task.

In [ ]:
resp = agent.resume("Tell me a one paragraph joke")
async for chunk in resp:
    print_chunk(chunk)


I understand you'd like a joke, but let me first get the current time information you requested. I need to know:

1. What time format would you prefer? (For example: "h:mm A" for 12-hour format, "YYYY-MM-DD HH:mm:ss" for full date and time, etc.)
2. What timezone should I use? (For example: "America/New_York", "Europe/London", "Asia/Tokyo", etc.)
Tool: "user_input"


### c) Resume with input

Provide required response and continue with original task.

In [ ]:
resp = agent.resume("Use SGT")
async for chunk in resp:
    print_chunk(chunk)


Thank you! I'll get the current time in Singapore Time (SGT). Let me use a standard format that shows both date and time:
Tool: "current_time"
	Input: format = YYYY-MM-DD HH:mm:ss
	Input: timezone = Asia/Singapore


---

## Scenario 4: Undo last state

In this scenario, the user undo the first message and provide a new one.

### a) Start Agent


In [2]:
agent = ToolUseAgent(
    agent_name="ToolUseAgent",
    llm_config=llm_config,
    aggregated_client=aggregated_client,
    monologue=monologue,
)

# Start the agent

user_message = "What is the current time? Please ask if you need more information."

resp = agent.start(user_message=user_message)
async for chunk in resp:
    print_chunk(chunk)

I'll help you get the current time. To provide you with the most accurate information, I need to know what format and timezone you'd like the time displayed in.
Tool: "user_input"


In [4]:
state =  agent.undo()
print(f"current state is {state}")

WARNING  MessageStore: No messages to insert. path=/workspaces/gai-sdk/test/integrationtest/tmp/monologue.json

current state is IS_TOOL_CALL


In [5]:
resp = agent.resume("Tell me a one paragraph joke")
async for chunk in resp:
    print_chunk(chunk)


Hi there! I'm ToolUseAgent, and I'd be happy to tell you a joke! Here's one for you:

A programmer's wife asks him to go to the store and buy a gallon of milk, and if they have eggs, buy a dozen. So the programmer goes to the store, sees they have eggs, and comes home with twelve gallons of milk. His wife asks, "Why did you buy so much milk?" The programmer replies, "They had eggs!" She facepalms and says, "I should have been more specific with my boolean logic." The programmer nods and says, "Well, technically I executed your instructions perfectly - you said 'if they have eggs, buy a dozen,' so I bought a dozen gallons of milk!" His wife sighs and mutters, "Next time I'm writing the grocery list in pseudocode."

Hope that gave you a chuckle!


Confirm by checking the monologue.

In [6]:
agent.monologue.list_chat_messages()

[{'role': 'user',
  'content': '\n            Your name is ToolUseAgent within the context of this conversation and you will always respond as such.\n            Do not refer to yourself as an AI or a bot or confuse your name with other agents.\n           \n            You may respond to my following message using the context you have learnt.\n            Tell me a one paragraph joke\n            \n            You may ask me for more information if you need to clarify my request but ask just enough to get the information you need to get started.\n            If you need to ask for more information, please use the "user_input" tool to get the information from me.\n            Please be very specific about what you need from me. Don\'t end with "I need some additional information" or similar phrases as you need to be specific about what you need.\n            '},
 {'role': 'assistant',
  'content': [{'citations': None,
    'text': 'Hi there! I\'m ToolUseAgent, and I\'d be happy to tell 